# Question Answering Challenge

### Using a Wikipedia dump for offline retrieval

As specified in https://en.wikipedia.org/wiki/Wikipedia:Database_download, we download the Polish Wikipedia dump from https://dumps.wikimedia.org/.

In [ ]:
import requests
from tqdm import tqdm

url = 'https://dumps.wikimedia.org/plwiki/latest/plwiki-latest-pages-articles.xml.bz2'

response = requests.get(url, stream=True)
if response.status_code == 200:
    file_name = 'plwiki-latest-pages-articles.xml.bz2'
    total_size = int(response.headers.get('content-length', 0))
    chunk_size = 1024  # 1 KB
    with open(file_name, 'wb') as file, tqdm(
        desc=file_name,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))
    print(f'Wikipedia dump downloaded successfully: {file_name}')
else:
    print(f"Failed to download the Wikipedia dump. HTTP Status Code: {response.status_code}")


### Extracting data from the Wikipedia dump using WikiExtractor

Run the following command to extract articles from the dump into an `extracted/` folder:
```bash
wikiextractor --json plwiki-latest-pages-articles.xml.bz2 -o extracted
```

### Processing the extracted data and indexing it into Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

INDEX_NAME = "wiki_index"

es = Elasticsearch("http://localhost:9200")
es.indices.delete(index=INDEX_NAME, ignore=[400, 404])

In [31]:
import os
import json
import logging
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

INDEX_NAME = "wiki_index"

INDEX_BODY = {
  "settings": {
    "analysis": {
      "analyzer": {
        "polish_stempel": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "polish_stop",
            "polish_stem"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "title": {
        "type": "text",
        "analyzer": "polish_stempel"
      },
      "content": {
        "type": "text",
        "analyzer": "polish_stempel"
      }
    }
  }
}



es = Elasticsearch("http://localhost:9200")

# UTWORZENIE INDEKSU (jeśli nie istnieje)
if not es.indices.exists(index=INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body=INDEX_BODY)
    # Wyłączamy odświeżanie i replikę na czas indeksowania (przyspieszy to import)
    es.indices.put_settings(
        index=INDEX_NAME,
        body={
            "index": {
                "refresh_interval": "-1",
                "number_of_replicas": 0
            }
        }
    )
    print(f"Created index '{INDEX_NAME}' with custom Polish analyzer and temporary settings.")

# DEFINICJA GENERATORA DOKUMENTÓW
def generate_actions(dump_dir, index_name):
    """
    Generator dla parallel_bulk. Dla każdego pliku wiki_... wyciąga artykuły,
    dzieli je na paragrafy i yielduje dokumenty do zindeksowania.
    """
    for root, _, files in os.walk(dump_dir):
        for file in files:
            if file.startswith('wiki_'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        article = json.loads(line)
                        title = article.get('title', '')
                        content = article.get('text', '')

                        paragraphs = content.split("\n")
                        for i, paragraph in enumerate(paragraphs):
                            paragraph = paragraph.strip()
                            if paragraph:
                                yield {
                                    "_index": index_name,
                                    "_source": {
                                        "title": title,
                                        "paragraph_number": i,
                                        "content": paragraph
                                    }
                                }

# WYWOŁANIE parallel_bulk DO MASOWEGO INDEKSOWANIA
actions = generate_actions("extracted", INDEX_NAME)

successes = 0
for ok, resp in parallel_bulk(
    client=es,
    actions=actions,
    thread_count=4,      # liczba wątków
    chunk_size=500,      # wielkość jednej paczki dokumentów
    max_chunk_bytes=10 * 1024 * 1024  # ~10 MB na paczkę
):
    if not ok:
        logging.error(f"Error indexing chunk: {resp}")
    else:
        successes += 1

print(f"Successfully processed {successes} chunks of data.")

# PRZYWRACANIE NORMALNYCH USTAWIEŃ
es.indices.put_settings(
    index=INDEX_NAME,
    body={
        "index": {
            "refresh_interval": "1s",
            "number_of_replicas": 1
        }
    }
)
print(f"Indexing complete. Restored normal settings for '{INDEX_NAME}'.")


Created index 'wiki_index' with custom Polish analyzer and temporary settings.
Processing file: extracted\AA\wiki_00
Processing file: extracted\AA\wiki_01
Processing file: extracted\AA\wiki_02
Processing file: extracted\AA\wiki_03
Processing file: extracted\AA\wiki_04
Processing file: extracted\AA\wiki_05
Processing file: extracted\AA\wiki_06
Processing file: extracted\AA\wiki_07
Processing file: extracted\AA\wiki_08
Processing file: extracted\AA\wiki_09
Processing file: extracted\AA\wiki_10
Processing file: extracted\AA\wiki_11
Processing file: extracted\AA\wiki_12
Processing file: extracted\AA\wiki_13
Processing file: extracted\AA\wiki_14
Processing file: extracted\AA\wiki_15
Processing file: extracted\AA\wiki_16
Processing file: extracted\AA\wiki_17
Processing file: extracted\AA\wiki_18
Processing file: extracted\AA\wiki_19
Processing file: extracted\AA\wiki_20
Processing file: extracted\AA\wiki_21
Processing file: extracted\AA\wiki_22
Processing file: extracted\AA\wiki_23
Processin

### Searching and answer extraction

In [32]:
from transformers import pipeline
from elasticsearch import Elasticsearch

# Inicjalizacja Elasticsearch i pipeline QA
es = Elasticsearch("http://localhost:9200")
#qa_pipeline = pipeline("question-answering", model="radlab/polish-qa-v2")
qa_pipeline = pipeline("question-answering", model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2", device=0)
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-roberta-large-v2") <- super bad
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-gpt2-xl") <- super bad as well

Some weights of the model checkpoint at henryk/bert-base-multilingual-cased-finetuned-polish-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


## Ollama requests

In [33]:
import requests
import json

def query_ollama(prompt, model="llama3.1", server_url="http://localhost:11434"):
    url = f"{server_url}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers, stream=True)
    
    if response.status_code == 200:
        # Przetwarzanie strumienia odpowiedzi
        result = ""
        for chunk in response.iter_lines():
            if chunk:
                data = json.loads(chunk.decode('utf-8'))
                if "response" in data:
                    result += data["response"]
                if data.get("done"):
                    break
        return result
    else:
        raise Exception(f"Błąd: {response.status_code}, {response.text}")


In [34]:
# Przykład użycia:
prompt = "Opisz funkcjonalność modelu LLama 3.1 w jednym zdaniu."
try:
    answer = query_ollama(prompt)
    print("Model odpowiedział:", answer)
except Exception as e:
    print("Wystąpił błąd:", e)

Model odpowiedział: Model Llama 3.1 jest dużą grupą neuronów sztucznych, która może wykonywać złożone zadania językowe, takie jak odpowiedzi na pytania, generowanie tekstu oraz tłumaczenie.


## Ultimate parse

In [39]:
import spacy
import re

nlp = spacy.load('pl_core_news_sm')

def extract_symbols(text):
    pattern = r'\b[A-Z][a-zA-Z]?\b'
    symbols = re.findall(pattern, text)
    return symbols

def is_important_token(token, text, symbols_set):
    return (token.pos_ in ["NOUN", "VERB", "NUM", "ADJ", "ADV", "DET"]
            and token.text not in symbols_set
            and not token.is_stop)

def get_keywords(question):
    doc = nlp(question)
    all_symbols = extract_symbols(question)
    symbols_set = {sym for sym in all_symbols if not nlp(sym)[0].is_stop}
    
    keywords = [
        [token.lemma_, token.text] for token in doc
        if is_important_token(token, question, symbols_set)
    ]
    return list(symbols_set), keywords

def extractentities(question):
    doc = nlp(question)
    entities = [(ent.text, ent.lemma_, ent.label_) for ent in doc.ents]
    return entities

def extract_and_remove_quotes(text):
    pattern = r"'(.*?)'"
    quotes = re.findall(pattern, text)
    modified_text = re.sub(pattern, '', text)
    return quotes, modified_text

def ultimate_parse(question):
    entities = extractentities(question)
    text = question
    for ent_text in [t for (t, _, _) in entities]:
        text = text.replace(ent_text, "")
    quotes, modified_text = extract_and_remove_quotes(text)
    lema_entities = [l for (_, l, _) in entities]
    symbols, tokens = get_keywords(modified_text)

    return quotes, lema_entities, symbols, tokens

In [42]:
print(ultimate_parse("W jakim państwie znajduje się Bombaj"))
print(ultimate_parse("Jak nazywa się pierwiastek o symbolu Bq"))
print(ultimate_parse("Ile pełnych tygodni ma rok kalendarzowy"))
print(ultimate_parse("W którym państwie rozpoczyna się akcja powieści 'W pustyni i w puszczy'?"))
print(ultimate_parse("Jak nazywał się sztywny kapelusz męski o zaokrąglonej główce i wąskim lekko uniesionym rondelku, modny w drugiej połowie XIX wieku"))
print(ultimate_parse("Jak nazywa się wypukła albo wklęsła powierzchnia cieczy w pobliżu ścianek naczynia"))
print(ultimate_parse("Jak brzmi ulubione powiedzonko klucznika Horeszków Gerwazego?"))
print(ultimate_parse("Który z bohaterów powieści 'Lalka' Bolesława Prusa jest kupcem?"))
question = "Którego gazu jest najwięcej w powietrzu?"#"Kto stworzył Tomb Raider?"
print(ultimate_parse(question))
# print(get_keywords(question))
# answer_question(question, es, INDEX_NAME)

([], ['Bombaj'], [], [['jaki', 'jakim'], ['państwo', 'państwie'], ['znajdować', 'znajduje']])
([], [], ['Bq'], [['nazywać', 'nazywa'], ['pierwiastek', 'pierwiastek'], ['symbol', 'symbolu']])
([], [], [], [['pełny', 'pełnych'], ['tydzień', 'tygodni'], ['rok', 'rok'], ['kalendarzowy', 'kalendarzowy']])
(['W pustyni i w puszczy'], [], [], [['państwo', 'państwie'], ['rozpoczynać', 'rozpoczyna'], ['akcja', 'akcja'], ['powieść', 'powieści']])
([], ['XIX wiek'], [], [['nazywać', 'nazywał'], ['sztywny', 'sztywny'], ['kapelusz', 'kapelusz'], ['Męski', 'męski'], ['zaokrąglony', 'zaokrąglonej'], ['główka', 'główce'], ['wąski', 'wąskim'], ['lekko', 'lekko'], ['unieść', 'uniesionym'], ['rondelke', 'rondelku'], ['modny', 'modny'], ['drugi', 'drugiej'], ['połowa', 'połowie']])
([], [], [], [['nazywać', 'nazywa'], ['wypuknąć', 'wypukła'], ['wklęsła', 'wklęsła'], ['powierzchnia', 'powierzchnia'], ['ciecz', 'cieczy'], ['pobliże', 'pobliżu'], ['ścianek', 'ścianek'], ['naczynie', 'naczynia']])
([], ['Hore

In [125]:
INDEX_NAME = "wiki_index"

# 2. Funkcja pobierająca akapity z ES
def retrieve_paragraphs(question, es_client, index_name, size=10):
    quotes, lema_entities, symbols, tokens = ultimate_parse(question)

    must_clauses = []
    for quote in quotes:
        must_clauses.append({"match_phrase": {"content": quote}})

    should_clauses = []
    for entity in lema_entities:
        # multi match with and condition and fuzziness
        should_clauses.append({
            "multi_match": {
                "query": entity,
                "fields": ["title^2", "content"],
                "operator": "and",
                "fuzziness": "AUTO",
                "boost": 5.0
            }
        })
    if symbols:
        should_clauses.append({
            "multi_match": {
                "query": " ".join(symbols),
                "fields": ["title", "content"],
                "fuzziness": "AUTO",
                "boost": 5.0
            }
        })
    for token_forms in tokens:
        should_clauses.append({
            # "multi_match": {
            #     "query": token,
            #     "fields": ["title", "content"],
            #     "operator": "and",
            #     "fuzziness": "AUTO",
            #     "boost": 2.0
            # }
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": " ".join(token_forms),
                            "fields": ["title", "content"],
                            "operator": "or",
                            "fuzziness": "AUTO",
                            "boost": 2.0
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        })
        
    # if tokens:
    #     should_clauses.append({
    #         "multi_match": {
    #             "query": " ".join(tokens),
    #             "fields": ["title", "content"],
    #             "minimum_should_match": "75%",
    #             "prefix_length": 2,
    #             "fuzziness": "2",
    #             "boost": 1.0
    #         }
    #     })

    query = {
        "query": {
            "bool": {
                "must": must_clauses,
                "should": should_clauses,
                "minimum_should_match": "40%"
            }
        },
        "size": size
    }

    response = es_client.search(index=index_name, body=query)

    paragraphs = []
    for hit in response["hits"]["hits"]:
        paragraphs.append(hit["_source"]["content"])
    return paragraphs

In [127]:
q1 = "Jak nazywał się naukowiec, który wynalazł telefon?"
q2 = "Którzy rzemieślnicy występują w ostatniej powieści Witkacego?"
q3 = "Co budował w` Egipcie inżynier Tarkowski, ojciec Stasia?"
q_bombaj = "W którym państwie jest Bombaj?"

q_bekeryl = "Pierwiastek symbolu Bq?"

parsed = ultimate_parse(q_bekeryl)
print(parsed)
paragraphs = retrieve_paragraphs(q_bekeryl, es, INDEX_NAME)
for i, paragraph in enumerate(paragraphs, 1):
    print(f"{i}. {paragraph}\n")
#print(retrieve_paragraphs(q2, es, INDEX_NAME))
#print(retrieve_paragraphs(q3, es, INDEX_NAME))

([], [], ['Bq'], [['pierwiastek', 'Pierwiastek'], ['symbol', 'symbolu']])
1. .bq – planowana domena internetowa przypisana do "Bonaire, Sint Eustatius i Saby".

2. Jednostką radioaktywności w układzie SI jest bekerel (Bq), 1 Bq = 1 rozpad na sekundę. Dawniej używaną, obecnie niezalecaną jednostką był kiur (Ci), 1 Ci = 3,7 formula_1 1010 Bq.

3. Jednostką aktywności jest bekerel, Bq:

4. W układzie SI bekerel (Bq)

5. W rzece naturalnie występują izotopy uranu-238, toru-232 i potasu-40, stężenie ich wynosi: od 6,18 do 68,76 Bq/kg dla U-238, od 8,12 do 89,28 Bq/kg dla Th-232 oraz od 99,01 do 312,16 Bq/kg dla K-40.

6. Koncentracja radonu w atmosferze jest zwykle mierzona w bekerelach na metr sześcienny (Bq/m³). Typowa koncentracja radonu wewnątrz budynków wynosi 100 Bq/m³ oraz 10–20 Bq/m³ na przestrzeniach otwartych. W USA stężenia radonu często są mierzone w pikokiurach na litr (pCi/l), będących w proporcji do berkeli 1 pCi/l = 37 Bq/m³.

7. Stężenia radonu mogą być bardzo różne w zależ

In [128]:
def get_best_answer_ollama(question, paragraphs):
    # Zapytanie do modelu LLama
    prompt = ""

    for i, paragraph in enumerate(paragraphs):
        prompt += f"\n\n{i+1}: {paragraph}"
    try:        
        prompt += "Powyżej znajduje się kilka możliwych kontekstów które mogą pomóc udzielić odpowiedzi na pytanie. "
        prompt += "Każdy w osobnym akapicie. Na ich podstawie proszę odpowiedzieć na pytanie. "
        prompt += "Uwaga, nie wszystkie konteksty pasują do zadanego pytania. "
        prompt += "Jeśli żaden kontekst nie pasuje, sam podaj odpowiedź. "
        prompt += "Jeśli to możliwe - odpowiedz jednym słowem, jeśli nie, bardzo krótko, w mniej niż 5 słowach. "
        prompt += "Nie wyjasniaj odpowiedzi ani sam nie dodawaj żadnych dodatkowych informacji. "
        prompt += "Nie pisz również którego akapitu użyłeś. Chcę tylko odpowiedź na pytanie."
        prompt += f"\n\nPytanie: {question}"
        if question.lower().startswith("czy"):
            question += " Odpowiedz koniecznie jednym słowem: tak/nie."
        answer = query_ollama(prompt)
        return answer
    except Exception as e:
        print("Wystąpił błąd:", e)
        return None

In [129]:
# test:
question = "Kto napisał Trylogię?"
paragraphs = ["Trylogia to cykl powieści fantasy autorstwa J.R.R. Tolkiena.", "Trylogia to cykl powieści autorstwa Jana Kowalskiego."]

answer = get_best_answer_ollama(question, paragraphs)
print("Model odpowiedział:", answer)

Model odpowiedział: J.R.R. Tolkien


In [130]:

# 3. Funkcja, która spośród pobranych paragrafów wybiera najlepszą odpowiedź
def get_best_answer(question, paragraphs):
    """
    Wywołuje pipeline QA dla każdego paragrafu i zwraca najlepszą odpowiedź wraz z wynikiem.
    """
    def update_best(result, current_best):
        """Aktualizuje najlepszy wynik i odpowiedź."""
        if result["score"] > current_best["score"]:
            return {"answer": result["answer"], "score": result["score"], "context": result["context"]}
        return current_best

    # Inicjalizacja najlepszych wyników dla pytania i jego słów kluczowych
    best_result = {"answer": None, "score": float("-inf"), "context": None}
    #best_result_kw = {"answer": None, "score": float("-inf"), "context": None}

    for paragraph in filter(str.strip, paragraphs):
        result = qa_pipeline(question=question, context=paragraph)
        result["context"] = paragraph
        #print('answer:', result["answer"])
        #print('score:', result["score"])
        best_result = update_best(result, best_result)

        #kw_result = qa_pipeline(question=get_keywords(question), context=paragraph)
        #kw_result["context"] = paragraph
        #best_result_kw = update_best(kw_result, best_result_kw)

    # if best_result["context"] and best_result_kw["context"]:
    #     combined_context = best_result["context"] + best_result_kw["context"]
    #     combined_result = qa_pipeline(question=question, context=combined_context)
    #     combined_result["context"] = combined_context
    #     best_result = update_best(combined_result, best_result)

    # elif best_result_kw["context"]:
    #     best_result = best_result_kw

    #print("best context:", best_result["context"])
    return best_result["answer"], best_result["score"]


In [131]:
# 4. Przykładowe wywołanie

def answer_question(question, es, index_name):
    paragraphs = retrieve_paragraphs(question, es, index_name, size=8)
    # print(f"Znaleziono {len(paragraphs)} pasujących paragrafów.")
    best_answer, best_score = get_best_answer(question, paragraphs)
    ollama_answer = get_best_answer_ollama(question, paragraphs)
    
    print(f"Pytanie: {question}")
    print(f"Najlepsza odpowiedź: {best_answer}")
    print(f"Score: {best_score}")
    print(f"Odpowiedź z LLamy: {ollama_answer}")

In [132]:
question = "Którego gazu jest najwięcej w powietrzu?"#"Kto stworzył Tomb Raider?"
# print(get_keywords(question))
answer_question(question, es, INDEX_NAME)


Pytanie: Którego gazu jest najwięcej w powietrzu?
Najlepsza odpowiedź: Wentylator
Score: 0.6189001202583313
Odpowiedź z LLamy: Azot.


In [133]:
question = "Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?"
answer_question(question, es, INDEX_NAME)

question = "Czy w państwach starożytnych powoływani byli posłowie i poselstwa?"
paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=8)
question_tuned = "Czy w starożytności powoływani byli posłowie i poselstwa? Tak czy nie?"
best_answer, best_score = get_best_answer(question_tuned, paragraphs)
ollama_answer = get_best_answer_ollama(question_tuned, paragraphs)

print(f"Pytanie: {question}")
print(f"Najlepsza odpowiedź: {best_answer}")
print(f"Score: {best_score}")
print(f"Odpowiedź z LLamy: {ollama_answer}")

Pytanie: Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?
Najlepsza odpowiedź: Lary Croft
Score: 0.8891009092330933
Odpowiedź z LLamy: Lara Croft
Pytanie: Czy w państwach starożytnych powoływani byli posłowie i poselstwa?
Najlepsza odpowiedź: nietykalności
Score: 0.18533863127231598
Odpowiedź z LLamy: Tak.


In [136]:
question = "Ile pełnych tygodni ma rok kalendarzowy?"
answer_question(question, es, INDEX_NAME)
print('-------------------')

question = "Co ma symbol Bq?"
answer_question(question, es, INDEX_NAME)

Pytanie: Ile pełnych tygodni ma rok kalendarzowy?
Najlepsza odpowiedź: 52
Score: 0.7411821484565735
Odpowiedź z LLamy: 52
-------------------
Pytanie: Co ma symbol Bq?
Najlepsza odpowiedź: bekerel
Score: 0.5227404236793518
Odpowiedź z LLamy: Bekerel.


### Evaluation

In [138]:
from difflib import SequenceMatcher
import re

def levenshtein_distance(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

def is_textual_match(pred, gold, threshold=0.5):
    return levenshtein_distance(pred.lower(), gold.lower()) >= threshold

def is_numerical_match(pred, gold):
    pred_num = re.search(r"\d+", pred)
    gold_num = re.search(r"\d+", gold)
    if pred_num and gold_num:
        return pred_num.group() == gold_num.group()
    return False

def evaluate(in_file, expected_file):
    with open(in_file, 'r', encoding='utf-8') as file_in, open(expected_file, 'r', encoding='utf-8') as file_expected:
        questions = [line.strip() for line in file_in]
        gold_answers = [line.strip() for line in file_expected]
    
    total = 200
    print(f"Total questions: {total}")
    correct = 0

    for index, (question, gold) in enumerate(zip(questions, gold_answers)):
        if index >= total:
            # My machine cannot handle more
            break
        paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=8)
        print(f'paragraphs: {paragraphs}')
        #best_answer, best_score = get_best_answer(question, paragraphs)
        #print(f"keywords: {get_keywords(question)}")
        #print(f"Q: {question}\nA: {best_answer}\nExpected: {gold}\n\n")
        #print(f"Score: {best_score}\n\n")
        # if best_answer is None:
        #     continue

        best_answer = get_best_answer_ollama(question, paragraphs)
        print(f"Q: {question}\nA: {best_answer}\nExpected: {gold}")
        if best_answer is None:
            print("No answer found!\n")
            continue
        if is_numerical_match(best_answer, gold) or is_textual_match(best_answer, gold):
            print("Correct!\n")
            correct += 1
        else:
            print("Incorrect!\n")

    print(f"Correct: {correct}")
    accuracy = float(correct) / float(total)
    print(f"Accuracy: {accuracy:.2%}")

DEV_0_IN = "data/dev-0/in.tsv"
DEV_0_EXPECTED = "data/dev-0/expected.tsv"
evaluate(DEV_0_IN, DEV_0_EXPECTED)

# Jak nazywał się fizyk, który w 1876 r. wynalazł telefon?
# Którzy rzemieślnicy występują w ostatniej powieści Witkacego?
# Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?
# Kto jest autorem obrazu namalowanego w 1892 r. „Kobiety z Tahiti”?
# W którym wieku powstał na Widawie most św. Karola?
# Które wrota znajdują się w tytule filmu Romana Polańskiego?


Total questions: 200
paragraphs: ['Dyftongi greckie dzielą się na właściwe i niewłaściwe (wymawiane jako pojedyncza samogłoska). Właściwe to , , , , , , . Niewłaściwe to , , , . Jeśli wyraz piszemy wielką literą, we wszystkich dyftongach duża jest tylko pierwsza litera składowa. Znak diakrytyczny przy dyftongach niewłaściwych , , pochodzi od małej litery jota i nazywa się "iota subscriptum". Jeśli dyftongi te piszemy wielką literą, pierwszą literę dyftongu zapisujemy jako wielką literę, jotę zapisujemy zaś obok, ale nie wymawiamy (nazywając ją wtedy "iota adscriptum", tj. , , ). Ułatwieniem w określeniu czy jest dyftongiem właściwym, czy nie, jest fakt, że w przydech i akcent w dyftongach właściwych stoi na drugiej głosce, w niewłaściwych zaś na pierwszej, np. dyftong właściwy: -, dyftong niewłaściwy -.', 'Alfabet grecki używany był także do zapisu liczb. Pierwsze dziewięć liter alfabetu odpowiadało liczbom od 1 do 9, kolejne dziewięć oznaczało wielokrotności liczby 10. Analogicznie, n